In [ ]:
import requests
from bs4 import BeautifulSoup
import re
from urllib.parse import urljoin, urlparse, urlsplit
import time
from urllib.robotparser import RobotFileParser
# visits up to 100 pages within the MIT domain,
# adhering to robots.txt rules,
#logs the canonical URLs of visited pages and their outgoing links to a text file, while pausing for 5 seconds between page visits.

In [ ]:
#Task 1  #visit webpages in tht domain
start_url = 'http://www.mit.edu'
urls_to_visit = set([start_url])
visited_urls = set()
domain_pattern = re.compile('^https?://(www\.)?mit\.edu')# filter out URLs from other domains.
html_pages_visited = 0
output_file = open('link.txt', 'w')
#Task 8
rp = RobotFileParser()# to communicate with web crawlers or robots
rp.set_url('https://www.mit.edu/robots.txt')
rp.read()
#Task 2
while urls_to_visit and html_pages_visited < 100:#till 100 webpages
    url = urls_to_visit.pop()
    if url in visited_urls:
        continue
    # Task 8
    if not rp.can_fetch('*', url):
        continue
    try:
        response = requests.get(url)
    except requests.exceptions.RequestException:
        continue
    content_type = response.headers.get('Content-Type')
    if content_type is not None:
        #Task 3:Detect whether a page you visit is an HTML doc or not?
        if 'text/html' in content_type:#whether html or not
            visited_urls.add(url)
            html_pages_visited += 1
            soup = BeautifulSoup(response.text, 'html.parser')#asy parsing and navigation of the HTML
            links = soup.find_all('a')#returns a list of tag objects representing the links present on the page.
            #Task 4: Detecting all outgoing links on each HTML page
            for link in links:
                href = link.get('href')#href variable will contain the URL or link destination.
                if href is not None:
                  #join base URL (url) with the relative URL extracted from the href
                    #Task 4 p2: This will require you to convert some links to a canonical form
                    href = urljoin(url, href.split('#')[0])#  remove  fragment identifier#section
                    parsed_href = urlparse(href)
                    if parsed_href.netloc == 'www.mit.edu' or parsed_href.netloc == 'mit.edu':#only links within the mit.edu domain are processed.
                        #Task 4: Follow only links to pages you have not yet visited.
                        if href not in visited_urls:
                            urls_to_visit.add(href)
                        #Task 5: For every web page you visit, write its canonical URL and the canonical URLs of each outgoing link to an HTML separated by a single space, to a plain text output file, with one line per visited page.
                        output_file.write(f'{urljoin(url, urlsplit(url).path)} {href}\n')
            #Task 5: For every web page you visit, write its canonical URL and the canonical URLs of each outgoing link to an HTML separated by a single space, to a plain text output file, with one line per visited page.
            output_file.write(f'{urljoin(url, urlsplit(url).path)}\n')
        #Task 3:Detect whether a page you visit is a PDF using the appropriate Content-Type HTTP header.
        elif 'application/pdf' in content_type:
            #Task 5: For every web page you visit, write its canonical URL and the canonical URLs of each outgoing link to PDF file separated by a single space, to a plain text output file, with one line per visited page.
            output_file.write(f'{urljoin(url, urlsplit(url).path)}\n')
    #Task 3:Detect whether a page you visit is something else using the appropriate Content-Type HTTP header.
    else:
        #Task 5: For every web page you visit, write its canonical URL and the canonical URLs of each outgoing link separated by a single space, to a plain text output file, with one line per visited page.
        output_file.write(f'{urljoin(url, urlsplit(url).path)}\n')
    visited_urls.add(url)
    #Task 8: Your crawler must visit at most one page per five seconds.
    time.sleep(5)
output_file.close()

#Task 6
  #Put some thought into the pages you decide to visit, and the order in which you visit them.
    #When i was deciding which pages should i vissit and it what oorder we prioritize the following:
      #1. we should give respect for the web server's resources
      #2. Relevance of the pages
      #3. Quality of the pages
  #Is there any way links on web pages could cause your crawler to misbehave?
      # Large pages
      # if we have infinite loops
      # if there are Dynamic pages
      # if there are Malicious links